In [2]:
import numpy as np
from numpy.linalg import inv

# Ejemplo 

In [3]:
#matriz de restricciones
A = np.array([[15, -36, 30, -54, 39, -92, 54, -76, 75, 30, 54, -36, -18, 79, 0, 0, 0, 0, 0, 0], 
     [-41, -38, 90, 74, 56, 37, 32, 92, 32, 35, 57, 64, 98, -51, 0, 0, 0, 0, 0, 0], 
     [-48, 38, -18, 20, 89, 15, 56, -64, -78, 35, 65, 55, -87, -23, 0, 0, 0, 0, 0, 0],
     [-65, 36, 77, 49, 74, 68, 56, -91, 37, 78, 9, 63, -36, -63, 0, 0, 0, 0, 0, 0],
     [100, 93, 50, 86, 50, 81, 67, 52, 52, 87, 61, 72, 68, 97, 1, 0, 0, 0, 0, 0],
     [-23, -27, -69, 40, -93, 48, -11, -51, 64, -82, 0, 47, 65, 97, 0, 1, 0, 0, 0, 0],
     [90, 18, 51, -78, 2, 98, -11, 74, -1, -7, 45, 70, 88, 1, 0, 0, 1, 0, 0, 0],
     [-3, 95, -57, -18, -33, 100, -18, 85, 84, -17, 42, 29, -52, 74, 0, 0, 0, 1, 0, 0],
     [47, 24, -19, 89, -74, 77, 13, 60, 83, 66, -17, -24, -3, 58, 0, 0, 0, 0, 1, 0],
     [-75, 43, 46, 72, 21, -99, 97, 92, 82, -25, 41, 50, -5, 57, 0, 0, 0, 0, 0, 1]])

#coeficientes de la función objetivo
c = np.array([44,18,60,27,-79,-51,92,-78,-69,-35,26,-56,-10,-55,0,0,0,0,0,0])

#términos independientes de las restricciones
b = np.array([64, 537, 55, 292, 1017, 6, 441, 312, 381, 398])

# Fase I (problema artificial)
## Usamos Fase II para resolver el problema

**1. Inicialización**

In [ ]:
# formato fase 1

m=len(A) # num de restricciones
n=len(A[0]) # num de variables
# num de variables no basicas = n-m // num de variables basicas = m

id=np.identity(m) #matriz identidad 

ones=np.ones(m) #vector de 1

c_fase_1=np.zeros(n) 
c_fase_1 = np.insert(c_fase_1,n,ones,axis=0) # función objetivo a minimizar en la fase I

A_fase_1 = np.insert(A,n,id,axis=1) # matriz de restricciones con las variables artificiales

indices_basicas = np.array(range(n,n+m))
indices_no_basicas = np.array(range(n))
indices_inventadas = np.array(range(n,n+m))

filas=np.array(range(m))
B = A_fase_1[np.ix_(filas,indices_basicas)]
An = A_fase_1[np.ix_(filas,indices_no_basicas)]

cn = c_fase_1[indices_no_basicas]
cb = c_fase_1[indices_basicas]
B_inv = inv(B)

Xb=np.matmul(B_inv,b)

z=np.dot(cb,Xb)

#comprobar si todos los valores son mayores o iguales a 0, con lo que significa que es una SBF
if all(val>=0 for val in Xb):
    #good
    print('Good')
    


Good


**2. Identificación de SBF óptima y selección de variable de entrada:**

In [ ]:
# r = Cn - Cb * inv(B) * An
cb_x_invB=np.matmul(cb,B_inv) # Multiplicación de cb por la inversa de B
cb_x_invB_An=np.matmul(cb_x_invB,An) # Multiplicación del resultado anterior por An
r=cn-cb_x_invB_An

print(r)

# Selección de la variable de entrada usando la regla de Bland
# elegimos la variable no básica de entrada con el subindice más pequeño con coste negativo 
optimo = True
for i in range(len(r)):
    # Como ordenamos las variables no básicas por los subíndices, elegimos el primer valor negativo. Si no hay valores negativos, quiere decir que es optimo
    if r[i]<0:
        q=i #subindice de la variable de entrada
        optimo = False
        break

if optimo:
    print("SBF optimo")
    # Aquí en la fase I comprovaremos que ningun indices pertenece a una variable artificial y que la z sea cero
    if z>=0:
        print("No factible")
    else:
        for indice in indices_basicas[i]:
            if indice in indices_inventadas:
                print("Ns/Nc")

            

[   3. -246. -181. -280. -131. -333. -335. -173. -430. -200. -357. -390.
 -118. -326.   -1.   -1.   -1.   -1.   -1.   -1.]


**3. Cálculo de un DBF de descenso:**

In [ ]:
q=np.array([q])
Aq = A_fase_1[np.ix_(filas,q)]
q_int=int(q)

db=np.matmul(-B_inv,Aq)
db_min=np.copy(db)
for i in range(m):
    if db_min[i]>0:
        db_min[i]=-0.0001

if all(val>=0 for val in db):
    print('(PL) no acotado')
    #stop

db=np.matmul(-B_inv,Aq)
db_min=np.matmul(-B_inv,Aq)
for i in range(m):
    if db_min[i]>0:
        db_min[i]=-0.0001

if all(val>=0 for val in db):
    print('(PL) no acotado')
    #stop

C:\Users\34644\AppData\Local\Temp\ipykernel_18160\2961260600.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  q_int=int(q)


**4. Cálculo de la longitud de paso máximo y selección de la variable de sálida:**

In [ ]:
pretheta=np.empty(0)
for i in range(m):
    print(-Xb[i]/db_min[i])
    pretheta=np.insert(pretheta,i,-Xb[i]/db_min[i])

theta=np.min(pretheta)
theta_index=np.argmin(pretheta)


theta_num=float(theta)
theta_index_int=int(theta_index)
p=theta_index_int

**5. Actualizaciones y cambio de base:**

In [ ]:
#Xb
theta_x_db= theta_num*db

Xb_actual=np.zeros(m)

for i in range(m):
    #print(i)
    Xb_actual[i]=Xb[i]+theta_x_db[i]
    if Xb_actual[i]==0:
        Xb_actual[i]=theta_num

#Z
z_float=float(z)
z_actual_actualizacion=z_float+r[q_int]*theta_num

#Actualizar B y An

basica=indices_no_basicas[int(q_int)]
no_basica=indices_basicas[int(p)]
indices_basicas[int(p)]=basica
indices_no_basicas[int(q_int)]=no_basica

col_An = q_int  
col_B = int(p) 

An[:, col_An], B[:, col_B] = B[:, col_B].copy().reshape(-1), An[:, col_An].copy().reshape(-1)

indices_no_basicas_sort = np.argsort(indices_no_basicas)  # Get sorted index positions
sorted_indices_array = indices_no_basicas[indices_no_basicas_sort]  # Sorted index array
sorted_An = An[:, indices_no_basicas_sort]  # Reorder columns of An based on sorted indices

cn = c_fase_1[indices_no_basicas_sort]
cb = c_fase_1[indices_basicas]

#Xb comprovación

B_inv_2=inv(B)
Xb_actual_2=np.matmul(B_inv_2,b)

#Z comprovación

z_actual=np.dot(cb,Xb_actual_2)

#comprovaciones

z_actual==z_actual_actualizacion
Xb_actual==Xb_actual_2
z_actual_actualizacion<z_float

B An cb cn z Xb indices_basicas indices_no_basicas_sort